In [1]:
import os

folder_path = r'D:\CSE463 Homework 04\training images\melanoma'

image_count = sum([1 for file in os.listdir(folder_path) if file.lower().endswith(('.png', '.jpg', '.jpeg'))])

print(f"Number of images in the folder: {image_count}")


Number of images in the folder: 673


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = r"D:\CSE463 Homework 04\training images"
img_size = 224  

datagen = ImageDataGenerator(
    rescale=1.0 / 255, 
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 539 images belonging to 1 classes.
Found 134 images belonging to 1 classes.


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x) 
    x = Dense(256, activation='relu')(x) 
    x = Dense(1, activation='sigmoid')(x) 
    model = Model(inputs=base_model.input, outputs=x)
    return model


In [4]:
from tensorflow.keras.optimizers import Adam

def train_model(model_name, base_model_fn, img_size):
    print(f"Training {model_name}...")
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    model = build_model(base_model)

    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

 
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=5, 
        verbose=1
    )

    results = model.evaluate(val_data, verbose=1)
    print(f"{model_name} Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

    return model


In [6]:
# Train VGG16
from tensorflow.keras.applications import VGG16

vgg16_model = train_model("VGG16", VGG16, 224)


Training VGG16...
Epoch 1/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 1.0000 - loss: 0.0923

c:\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 172s 10s/step - accuracy: 1.0000 - loss: 0.0888 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 168s 10s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 157s 9s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 156s 9s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 157s 9s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00
VGG16 Validation Loss: 0.0, Validation Accuracy: 1.0


In [7]:
# Train ResNet50
from tensorflow.keras.applications import ResNet50

resnet50_model = train_model("ResNet50", ResNet50, 224)


Training ResNet50...
Epoch 1/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 126s 6s/step - accuracy: 0.9126 - loss: 0.2118 - val_accuracy: 1.0000 - val_loss: 9.7968e-04
Epoch 2/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 103s 6s/step - accuracy: 1.0000 - loss: 1.2929e-04 - val_accuracy: 1.0000 - val_loss: 4.1825e-05
Epoch 3/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 99s 6s/step - accuracy: 1.0000 - loss: 4.1742e-05 - val_accuracy: 1.0000 - val_loss: 6.3108e-05
Epoch 4/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 101s 6s/step - accuracy: 1.0000 - loss: 2.9396e-05 - val_accuracy: 1.0000 - val_loss: 7.0698e-05
Epoch 5/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 98s 6s/step - accuracy: 1.0000 - loss: 3.5429e-05 - val_accuracy: 1.0000 - val_loss: 3.8392e-05
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 1.0000 - loss: 3.9428e-05
ResNet50 Validation Loss: 3.885593105223961e-05, Validation Accuracy: 1.0


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Paths
train_dir = "D:/CSE463 Homework 04/training images"

# Parameters
img_size = (224, 224)
batch_size = 32
epochs = 5
learning_rate = 0.001

datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Rescale pixel values
    validation_split=0.2      # 20% validation split
)

# Load Training Data
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',     
    subset='training'
)

# Load Validation Data
val_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',      
    subset='validation'
)

# Load Pretrained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  

# Add Custom Classification Head
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dense(128, activation='relu')(x)  # Fully connected layer
output = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Create the Model
model = Model(inputs=base_model.input, outputs=output)

# Compile the Model
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Training MobileNetV2...")
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    steps_per_epoch=train_data.samples // batch_size,
    validation_steps=val_data.samples // batch_size
)

model.save("melanoma_mobilenetv2.h5")
print("Model saved as melanoma_mobilenetv2.h5")

loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


Found 539 images belonging to 1 classes.
Found 134 images belonging to 1 classes.
Training MobileNetV2...
Epoch 1/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 14s 622ms/step - accuracy: 0.8023 - loss: 0.2874 - val_accuracy: 1.0000 - val_loss: 9.2304e-08
Epoch 2/5
 1/16 ━━━━━━━━━━━━━━━━━━━━ 6s 402ms/step - accuracy: 1.0000 - loss: 1.2168e-08

c:\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 1.0000 - loss: 1.2168e-08 - val_accuracy: 1.0000 - val_loss: 7.1618e-08
Epoch 3/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 532ms/step - accuracy: 1.0000 - loss: 1.9375e-09 - val_accuracy: 1.0000 - val_loss: 9.6163e-09
Epoch 4/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - accuracy: 1.0000 - loss: 3.8992e-10 - val_accuracy: 1.0000 - val_loss: 9.2343e-09
Epoch 5/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 531ms/step - accuracy: 1.0000 - loss: 2.8436e-10 - val_accuracy: 1.0000 - val_loss: 6.1743e-09


Model saved as melanoma_mobilenetv2.h5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 339ms/step - accuracy: 1.0000 - loss: 4.3046e-09
Validation Loss: 5.912384803963278e-09
Validation Accuracy: 1.0


In [ ]:
vgg16_model = tf.keras.models.load_model('path_to_vgg16_model')  # TensorFlow SavedModel format
resnet50_model = tf.keras.models.load_model('path_to_resnet50_model')  # TensorFlow SavedModel format
mobilenetv2_model = tf.keras.models.load_model('path_to_mobilenetv2_model')  # TensorFlow SavedModel format


In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import os
import numpy as np

# Define model filenames
model_files = ["vgg16_melanoma.h5", "resnet50_melanoma.h5", "melanoma_mobilenetv2.h5"]

# Verify model files exist
missing_files = [file for file in model_files if not os.path.exists(file)]
if missing_files:
    print(f"Missing model files: {missing_files}")
    print("Please ensure the models are trained and saved correctly.")
    exit()

# Load models
vgg16_model = tf.keras.models.load_model("vgg16_melanoma.h5")
resnet50_model = tf.keras.models.load_model("resnet50_melanoma.h5")
mobilenetv2_model = tf.keras.models.load_model("melanoma_mobilenetv2.h5")

# Load dataset for evaluation
data_dir = "D:/CSE463 Homework 04/training images"  # Update with your dataset path
img_size = 224
batch_size = 32

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=False
)

# Extract images and labels for metrics evaluation
images = []
labels = []
for batch_images, batch_labels in dataset:
    images.append(batch_images.numpy())
    labels.append(batch_labels.numpy())
images = np.concatenate(images, axis=0)
labels = np.concatenate(labels, axis=0)

# Evaluate models
def evaluate_model(model, model_name):
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, predicted_classes)
    precision = precision_score(labels, predicted_classes, average="weighted", zero_division=0)
    recall = recall_score(labels, predicted_classes, average="weighted", zero_division=0)
    f1 = f1_score(labels, predicted_classes, average="weighted", zero_division=0)

    print(f"\nModel: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(labels, predicted_classes, zero_division=0))

# Evaluate each model
evaluate_model(vgg16_model, "VGG16")
evaluate_model(resnet50_model, "ResNet50")
evaluate_model(mobilenetv2_model, "MobileNetV2")


In [ ]:
'''
as we can see from the results, the MobileNetV2 model has the highest accuracy, precision, recall, and F1 score among the three models.
'''